# Markov Reward Process

![image.png](./image.png)

这里假设从 $s_1$ 出发，terminal state 是 $s_6$

In [ ]:
import numpy as np

P = [
    [0.9, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.6, 0.0, 0.4],
    [0.0, 0.0, 0.0, 0.0, 0.3, 0.7],
    [0.0, 0.2, 0.3, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
]

P = np.array(P)
rewards = [-1, -2, -2, 10, 1, 0]
gama = 0.5

def get_return(start_index, chain: list, gama):
    G = 0
    for i in reversed(range(start_index, len(chain))):
        G += gama * G + rewards[chain[i]-1]
    return G





